# Анализ пользовательских данных и заказов e-commerce платформы

<img src="https://storage.yandexcloud.net/klms-public/production/learning-content/152/1762/17923/51794/244290/2023_01_24_214337_negate.jpg" alt="Alt text" width="600">


### 1. Напишите запрос, который включит в результат две колонки с датами рождения из обеих таблиц, назвав их соответственно users_birth_date и couriers_birth_date. Включите также колонки с числом пользователей (users_count) и числом курьеров (couriers_count).

```sql
SELECT a.birth_date as users_birth_date,
       users_count,
       b.birth_date as couriers_birth_date,
       couriers_count
FROM   (SELECT birth_date,
               count(user_id) as users_count
        FROM   users
        WHERE  birth_date is not null
        GROUP BY birth_date) a full JOIN (SELECT birth_date,
                                                 count(courier_id) as couriers_count
                                          FROM   couriers
                                          WHERE  birth_date is not null
                                          GROUP BY birth_date) b using (birth_date)
ORDER BY users_birth_date, couriers_birth_date
```

### 2. Напишите запрос, который отберёт id первых 100 пользователей из таблицы `users` и объединит их со всеми наименованиями товаров из таблицы `products`.

```sql
SELECT user_id,
       name
FROM   (SELECT user_id
        FROM   users
        LIMIT 100) as t1
        CROSS JOIN products
ORDER BY user_id, name
```

### 3. Напишите запрос, который для каждого пользователя рассчитает следующие показатели, учитывая только неотменённые заказы:

- Общее число заказов — назовите колонку `orders_count`.
- Среднее количество товаров в заказе — назовите колонку `avg_order_size` (округлить до двух знаков после запятой).
- Суммарную стоимость всех покупок — назовите колонку `sum_order_value`.
- Среднюю стоимость заказа — назовите колонку `avg_order_value` (округлить до двух знаков после запятой).
- Минимальную стоимость заказа — назовите колонку `min_order_value`.
- Максимальную стоимость заказа — назовите колонку `max_order_value`.

```sql
SELECT user_id,
       count(order_price) as orders_count,
       round(avg(order_size), 2) as avg_order_size,
       sum(order_price) as sum_order_value,
       round(avg(order_price), 2) as avg_order_value,
       min(order_price) as min_order_value,
       max(order_price) as max_order_value
FROM   (SELECT user_id,
               order_id,
               array_length(product_ids, 1) as order_size
        FROM   (SELECT user_id,
                       order_id
                FROM   user_actions
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN orders using(order_id)) t2
    LEFT JOIN (SELECT order_id,
                      sum(price) as order_price
               FROM   (SELECT order_id,
                              product_ids,
                              unnest(product_ids) as product_id
                       FROM   orders
                       WHERE  order_id not in (SELECT order_id
                                               FROM   user_actions
                                               WHERE  action = 'cancel_order')) t3
                   LEFT JOIN products using(product_id)
               GROUP BY order_id) t4 using (order_id)
GROUP BY user_id
ORDER BY user_id limit 1000
```

### 4. Напишите запрос, который посчитает ежедневную выручку сервиса. Выручка определяется как стоимость всех реализованных товаров, содержащихся в заказах. Для этого используйте данные из таблиц `orders`, `products` и `user_actions`.


```sql
SELECT  date,
        sum(price) as revenue
FROM 
(SELECT order_id,
        date(time) as date,
        unnest(product_ids) as product_id
FROM   orders left JOIN user_actions using(order_id)
WHERE  order_id not in (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')) t1
LEFT JOIN products using (product_id)
GROUP BY date
ORDER BY date
```

### 5. Напишите запрос, который определит 10 самых популярных товаров, доставленных в сентябре 2022 года, используя данные из таблиц. Под популярностью товара понимается количество заказов, в которых он был доставлен, _с учётом только одной единицы товара_ за заказ. Выведите наименования товаров и количество заказов, в которых они встречались.

```sql
WITH September_orders as (
SELECT  DISTINCT order_id,  --DISTINCT тк учитываем только одну единицу товара
        unnest(product_ids) as product_id
FROM
   (SELECT order_id
    FROM courier_actions
    WHERE action = 'deliver_order' and 
    DATE_PART('year', time) = 2022 and
    DATE_PART('month', time) = 9) t1
LEFT JOIN orders USING(order_id)
)
SELECT  name, 
        count(*) as times_purchased
FROM September_orders LEFT JOIN products USING(product_id)
GROUP BY name
ORDER BY times_purchased desc
LIMIT 10
```

### 6. Напишите запрос, который заменит списки с id товаров в таблице orders на списки с наименованиями товаров из таблицы `products`. Новую колонку с наименованиями товаров назовите product_names.

```sql
SELECT order_id,
       array_agg(name) as product_names
FROM   (SELECT order_id,
               unnest(product_ids) as product_id
        FROM   orders) t1
    LEFT JOIN products using(product_id)
GROUP BY order_id 
```

### 7. На основе таблицы `orders` сформируйте новую таблицу с общим числом заказов по дням, исключив отменённые заказы. Затем расчитайте накопительную сумму числа заказов.

```sql
WITH Orders_by_day as (
SELECT  date(creation_time) as date,
        COUNT(DISTINCT order_id) as orders_count
FROM   orders
WHERE order_id not in (SELECT order_id 
                       FROM user_actions
                       WHERE action = 'cancel_order')
GROUP BY date
)
SELECT  date,
        orders_count,
        SUM(orders_count) OVER (ORDER BY date)::integer as orders_cum_count
FROM Orders_by_day
    
```

### 8. Для каждого пользователя в таблице user_actions посчитайте порядковый номер каждого заказа, исключив отменённые заказы. Затем рассчитайте, сколько времени прошло с момента предыдущего заказа для каждого пользователя.








```sql
WITH Orders_number as (
SELECT  user_id,
        order_id,
        time,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY time) as order_number,
        LAG(time) OVER (PARTITION BY user_id ORDER BY time) as time_lag
FROM user_actions
WHERE order_id not in (SELECT order_id 
                       FROM user_actions
                       WHERE action = 'cancel_order')
)
SELECT  *,
        time - time_lag as time_diff
FROM Orders_number
ORDER BY user_id, order_number
```

### 9. На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Учтите только тех пользователей, которые оформили более одного неотменённого заказа. Среднее время между заказами выразите в часах, округлив значения до целого числа.


```sql
WITH Orders_number as (
SELECT  user_id,
        order_id,
        time,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY time) as order_number,
        LAG(time) OVER (PARTITION BY user_id ORDER BY time) as time_lag
FROM user_actions
WHERE order_id not in (SELECT order_id 
                       FROM user_actions
                       WHERE action = 'cancel_order')),
Diff_time as (
SELECT  *,
        time - time_lag as time_diff
FROM Orders_number)

SELECT  DISTINCT user_id,
        ROUND(
            EXTRACT(epoch FROM AVG(time_diff) OVER (PARTITION BY user_id)
            ) / 3600
        )::integer AS hours_between_orders
FROM Diff_time
WHERE user_id in (SELECT user_id
                  FROM Orders_number
                  GROUP BY user_id
                  HAVING count(DISTINCT order_id) > 1)
ORDER BY user_id
LIMIT 1000 
```

### 10. На основе таблицы `orders` сформируйте новую таблицу с общим числом заказов по дням, исключив отменённые заказы. Затем рассчитайте скользящее среднее числа заказов, используя данные за три предыдущих дня.




```sql
WITH Orders_by_day as (
SELECT  date(creation_time) as date,
        COUNT(DISTINCT order_id) as orders_count
FROM   orders
WHERE order_id not in (SELECT order_id 
                       FROM user_actions
                       WHERE action = 'cancel_order')
GROUP BY date
)
SELECT  date,
        orders_count,
        ROUND(
            AVG(orders_count) OVER (ORDER BY date
                                    ROWS BETWEEN 3 PRECEDING and 1 PRECEDING),
        2) as moving_avg     
FROM Orders_by_day
    

```

### 11. Отметьте в отдельной таблице тех курьеров, которые доставили в сентябре 2022 года больше заказов, чем в среднем все курьеры. Используйте CASE, чтобы в отдельной колонке указать 1, если курьер доставил больше заказов, чем в среднем, и 0 в противном случае.


```sql
WITH September_orders as (
SELECT  courier_id,
        COUNT(DISTINCT order_id) as delivered_orders
FROM    courier_actions
WHERE order_id in (SELECT order_id 
                   FROM courier_actions
                   WHERE action = 'deliver_order')
      and DATE_PART('year', time) = 2022
      and DATE_PART('month', time) = 9
GROUP BY courier_id),
Avg_delivered AS (
SELECT  *,
        ROUND(
        AVG(delivered_orders) OVER (),
        2) as avg_delivered_orders
FROM September_orders
)
SELECT  *,
        CASE 
            WHEN delivered_orders > avg_delivered_orders THEN 1 
            ELSE 0
        END as is_above_avg
FROM Avg_delivered
    

```

### 12. По данным таблицы `user_actions` посчитайте число первых и повторных заказов на каждую дату. Для каждого пользователя первым заказом считается самый ранний заказ, все остальные заказы этого пользователя относятся к категории «Повторный».

### К полученному запросу добавьте расчёт доли первых и повторных заказов для каждого дня. Рассчитайте эти доли на основе общего числа заказов на каждую дату.

```sql
WITH Orders_numbers as (
SELECT  date(time),
        order_id,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY time) order_number
FROM    user_actions
WHERE order_id not in (SELECT order_id 
                       FROM user_actions
                       WHERE action = 'cancel_order')),
Orders_types as (                       
SELECT  *,
        CASE 
            WHEN order_number = 1 THEN 'Первый'
            ELSE 'Повторный'
        END as order_type
FROM Orders_numbers),
Total_orders as (
SELECT  date,
        order_type,
        COUNT(*) as orders_count
FROM Orders_types
GROUP BY date, order_type
ORDER BY date, order_type
)
SELECT  *,
        ROUND(
            orders_count::decimal /
                SUM(orders_count) OVER (PARTITION BY date),
        2) as orders_share
FROM Total_orders
```

### 13. Для каждой записи проставьте среднюю цену всех товаров. Затем рассчитайте среднюю цену товаров без учёта самого дорогого товара и поместите результат в отдельную колонку.


```sql  
SELECT  *,
        ROUND(AVG(price) OVER (), 2)  as avg_price,
        ROUND(
            AVG(price) FILTER (WHERE price <>
                                   (SELECT MAX(price) 
                                    FROM products))
                       OVER (),
        2) as avg_price_filtered
FROM products
ORDER BY price DESC, product_id
```

### 14. Для каждой записи в таблице `user_actions` с помощью FILTER посчитайте, сколько заказов сделал и сколько отменил каждый пользователь на момент совершения нового действия.

То есть, для каждого пользователя в каждый момент времени рассчитайте две накопительные суммы: число оформленных (created_orders) и число отменённых (canceled_orders) заказов. Если пользователь оформляет заказ, увеличивайте на 1 количество оформленных заказов; если отменяет — увеличивайте на 1 количество отменённых заказов.

На основе этих двух колонок рассчитайте показатель cancel_rate, т.е. долю отменённых заказов в общем количестве оформленных заказов.

В результате у вас должны получиться три новые колонки с динамическими показателями, которые изменяются во времени с каждым новым действием пользователя.


```sql
SELECT user_id,
       order_id,
       action,
       time,
       created_orders,
       canceled_orders,
       round(canceled_orders::decimal / created_orders, 2) as cancel_rate
FROM   (SELECT user_id,
               order_id,
               action,
               time,
               count(order_id) filter (WHERE action != 'cancel_order') OVER (PARTITION BY user_id
                                                                             ORDER BY time) as created_orders,
               count(order_id) filter (WHERE action = 'cancel_order') OVER (PARTITION BY user_id
                                                                            ORDER BY time) as canceled_orders
        FROM   user_actions) t
ORDER BY user_id, order_id, time limit 1000;
```

### 15. Из таблицы `courier_actions` отберите топ 10% курьеров по количеству доставленных за всё время заказов. Выведите id курьеров, количество доставленных заказов и порядковый номер курьера в соответствии с числом доставленных заказов.

У курьера, доставившего наибольшее число заказов, порядковый номер должен быть равен 1, а у курьера с наименьшим числом заказов среди топ 10% — числу, равному 10% от общего количества курьеров в таблице courier_actions.

```sql
with courier_count as (SELECT count(distinct courier_id)
                       FROM   courier_actions)
SELECT courier_id,
       orders_count,
       courier_rank
FROM   (SELECT courier_id,
               count(distinct order_id) as orders_count,
               row_number() OVER (ORDER BY count(distinct order_id) desc, courier_id) as courier_rank
        FROM   courier_actions
        WHERE  action = 'deliver_order'
        GROUP BY courier_id) as t1
WHERE  courier_rank <= round((SELECT *
                              FROM   courier_count)*0.1);
```

### 16. Отберите из таблицы `courier_actions` всех курьеров, которые работают в нашей компании 10 и более дней. Также рассчитайте, сколько заказов они уже успели доставить за всё время работы.

Период работы курьера рассчитывайте как разницу между первым действием курьера и временем последнего действия в таблице courier_actions. Текущей отметкой времени, относительно которой необходимо рассчитывать продолжительность работы курьера, считайте время последнего действия. Учтите только целые дни, прошедшие с момента первого выхода курьера на работу (часы и минуты не учитывайте).

```sql
SELECT courier_id,
       date_part ('day', last_date - first_date)::int as days_employed,
       delivered_orders FROM(SELECT courier_id,
                             count (order_id) filter (WHERE action = 'deliver_order') as delivered_orders,
                             min (time) as first_date,
                             (SELECT max(time)
                       FROM   courier_actions) as last_date
                      FROM   courier_actions
                      GROUP BY courier_id) t
WHERE  date_part ('day', last_date - first_date) >= 10
ORDER BY days_employed desc, courier_id;
```

### 17. На основе информации в таблицах `orders` и `products` рассчитайте стоимость каждого заказа, ежедневную выручку сервиса и долю стоимости каждого заказа в ежедневной выручке, выраженную в процентах. В результат включите следующие колонки:

- id заказа
- Время создания заказа
- Стоимость заказа
- Выручка за день, в который был совершён заказ
- Доля стоимости заказа в выручке за день, выраженная в процентах

```sql
SELECT order_id,
       creation_time,
       order_price,
       sum(order_price) OVER(PARTITION BY date(creation_time)) as daily_revenue,
       round(100 * order_price::decimal / sum(order_price) OVER(PARTITION BY date(creation_time)),
             3) as percentage_of_daily_revenue
FROM   (SELECT order_id,
               creation_time,
               sum(price) as order_price
        FROM   (SELECT order_id,
                       creation_time,
                       product_ids,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t3
            LEFT JOIN products using(product_id)
        GROUP BY order_id, creation_time) t
ORDER BY date(creation_time) desc, percentage_of_daily_revenue desc, order_id;
```

### 18. На основе информации в таблицах `orders` и `products` рассчитайте ежедневную выручку сервиса и отразите её в колонке `daily_revenue`. Затем посчитайте ежедневный прирост выручки. Отразите прирост выручки как в абсолютных значениях, так и в процентах относительно предыдущего дня.

Для самого первого дня укажите прирост равным 0 в обеих колонках. При проведении расчётов отменённые заказы не учитывайте. 

```sql
SELECT date,
       sum (price) as daily_revenue,
       coalesce (round (sum (price) - lag (sum (price), 1) OVER (ORDER BY date), 1), 0) as revenue_growth_abs,
       coalesce (round (100*sum (price)::decimal / lag (sum (price), 1) OVER (ORDER BY date) - 100, 1), 0) as revenue_growth_percentage
FROM   (SELECT order_id,
               creation_time::date as date,
               unnest(product_ids) as product_id
        FROM   orders
        WHERE  order_id not in (SELECT order_id
                                FROM   user_actions
                                WHERE  action = 'cancel_order')) t1
    LEFT JOIN products using(product_id)
GROUP BY date
ORDER BY date;
```